In [5]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
#TR 10/24/2024

# Import ANIMALSHELTER Class
from crud_ops import AnimalShelter

############################ Data Manipulation / Model############################ 
# MongoDB username and password
username = "aacuser"
password = "Tre1990"

# Initialize AnimalShelter class for database operations
db = AnimalShelter(username, password)

# Function to fetch data based on rescue type
def fetch_data(rescue_type):  #TR 10/24/2024
    query = {}
    if rescue_type == 'Water Rescue':
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == 'Mountain or Wilderness Rescue':
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == 'Disaster or Individual Tracking':
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    return db.read(query)

# Fetch initial data
data = fetch_data('Reset')
if data['status'] == 'success' and data['data']:  #TR 10/24/2024
    df = pd.DataFrame.from_records(data['data'])
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
else:
    print(f"Error fetching data: {data.get('message', 'Unknown error')}")
    df = pd.DataFrame()

########################## Dashboard Layout / View##########################
# Initialize the Dash app
app = JupyterDash(__name__)

# Add the Grazioso Salvare logo image
image_filename = '/home/tremainerayne_snhu/Desktop/Grazioso Salvare Logo.png'
try:
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()
except Exception as e:
    print(f"Error encoding image: {e}")
    encoded_image = None

# Define the layout for the dashboard
app.layout = html.Div([
    html.Center(html.B(html.H1('Tremaine Rayner'))),
    html.Hr(),
    html.Img(src=f'data:image/png;base64,{encoded_image}') if encoded_image else html.Div("Image not found"),
    html.Hr(),
    dcc.Dropdown(
        id='rescue-type-dropdown',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',
        placeholder="Select a rescue type"
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action='native',
        sort_action='native', 
        row_selectable='single',
        selected_rows=[],
        page_size=10,
        style_table={'height': '300px', 'overflowY': 'auto'},
        style_cell={'textAlign': 'left'},
    ),
    html.Br(),
    html.Hr(),  #TR 10/24/2024
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

############################################## Interaction Between Components / Controller##############################################
# Callback to update the data based on rescue type selection
@app.callback(
    [Output('datatable-id', 'data'),
     Output('graph-id', 'children'),
     Output('map-id', 'children')],
    [Input('rescue-type-dropdown', 'value'),
     Input('datatable-id', 'derived_virtual_selected_rows')])
def update_dashboard(rescue_type, selected_rows):
    data = fetch_data(rescue_type)
    if data['status'] == 'success' and data['data']:
        dff = pd.DataFrame.from_records(data['data'])
        if '_id' in dff.columns:
            dff.drop(columns=['_id'], inplace=True)
    else:
        dff = pd.DataFrame()

    # Update DataTable
    table_data = dff.to_dict('records')  #TR 10/24/2024

    # Update Pie Chart
    if not dff.empty:
        fig = px.pie(dff, names='breed', title='Animal Breeds')
        graph = dcc.Graph(figure=fig)
    else:
        graph = html.Div("No data available.")

    # Update Map
    if not dff.empty and selected_rows is not None and len(selected_rows) > 0:
        if len(selected_rows) > 0 and selected_rows[0] < len(dff):
            row = selected_rows[0]
            if 'location_lat' in dff.columns and 'location_long' in dff.columns:
                lat = dff.iloc[row]['location_lat']
                long = dff.iloc[row]['location_long']
                name = dff.iloc[row]['name']
                map_component = [
                    dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, long], zoom=10, children=[
                        dl.TileLayer(id="base-layer-id"),
                        dl.Marker(position=[lat, long], children=[
                            dl.Tooltip(dff.iloc[row]['breed']),
                            dl.Popup([html.H1(name), html.P(dff.iloc[row]['breed'])])
                        ])
                    ])
                ]
            else:
                map_component = html.Div("Location data not available.")  
        else:                                                                    #TR 10/24/2024
            map_component = html.Div("No map data available.")
    else:
        map_component = html.Div("No map data available.")

    return table_data, graph, map_component

# Callback to highlight selected column
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    if selected_columns is None:
        selected_columns = []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'  #TR 10/24/2024
    } for i in selected_columns]

# Run the app
app.run_server(debug=True)


HOST: nv-desktop-services.apporto.com, PORT: 31462
Dash app running on http://127.0.0.1:13293/
